# <center> L2-regularized Logistic Least Squares Regression </center>

</br>
On this exercise, you are going to apply what you learn from the `numpy` tutorial in the implementation of L2-regularized Logistic Least Squares Regression (LLSR). I will provide you the formula by now (you can do it yourself after the next lecture!!!), first you should use pens and papers to vectorize them. Then you implement the full of the classifier based on your vectorized version.

<img src="../Images/LogisticRegression.png" style="width:298px;height:275px">

</br>
L2-regularized Logistic Least Squares Regression is similar to the standard Logistic Regression: It is a binary classifier containing only one layer, mapping the input features to only one output using sigmoid function. The differents here are two things: 
* Instead of the _binary crossentropy error_ for the loss, it uses the _squared error_.
* It is applied the L2-regularization.

Note that we will do an SGD training for this exercise. More specifically:
* There are $m$ data instance on the training set, each has $n$ input features. 
* $x_{i}^{(j)}$ denotes the $i^{th}$ input feature of the $j^{th}$ data instance.
* $y^{(j)}$ denotes the binary label ($0$ or $1$) of the $j^{th}$ data instance.
* $w_{i}$ denotes the weight connecting the $i^{th}$ input feature to the output.
* $b$ is the bias of the Logistic Least Squares Regression.

So the steps of an unvectorized version are:
* The weights are initialized using Xavier Initialization, the bias can be initialized as 0.
* Train over 5 epochs, each epoch we do those steps:
  *  Loop over every data instance $x^{(j)}$:
     * Calculate the output of the LLSR: $o^{(j)} = \sigma(\sum_{i=1}^{n} w_ix_i^{(j)} + b)$
     * Calculate the cost: squared error $c^{(j)} = (y^{(j)} - o^{(j)})^2$
     * The final loss function is L2-regularized: $l^{(j)} = \frac{1}{2}c^{(j)} + \frac{\lambda}{2}\sum_{i=1}^{n}w_i^2$. 
     * Update the weights: 
         * Loop over every weight $w_i$ and update once at a time: $w_i = w_i - \eta((o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})x_i^{(j)} + \lambda w_i)$
     * Update the bias: $b = b - \eta (o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})$
  *  Calculate the total loss (of the epoch): $L = \frac{1}{m}\sum_{j=1}^{m}l^{(j)}$. Print it out. 


The guideline is to avoid explicit for-loops. _Hint_: We cannot make the epoch loop disappears, but all other loops can be replaced by vectorization.

First, let's import numpy and math:

In [1]:
import numpy as np
import math


We will use LLSR for the MNIST_SEVEN task: predict a $128\times 128$-pixel image of a handwritten digit whether it is a "7" or not. This is a binary classification task. I did the data reading for you. There is 5000 images, I split the first 4000 images for training, 500 images for tuning, 500 images for test. On this exercise we do not need to tune anything, so we'd leave the tuning (called the _dev set_) untouch. The first field is the label ("0"-"9") of the image, the rest are the grayscale value of each pixel. 

In [2]:
data_path = "../Data/mnist_seven.csv"
data = np.genfromtxt(data_path, delimiter=",", dtype="uint8")
train, dev, test = data[:4000], data[4000:4500], data[4500:]

In [3]:
def normalize(dataset):
    X = dataset[:, 1:] / 255.     # Normalize input features
    Y = (dataset[:, 0] == 7) * 1  # Convert labels from 0-9 to Is7 (1) or IsNot7(0)
    return X.T,Y.reshape(1, -1)

In [4]:
X_train, Y_train = normalize(train)
print(X_train.shape)
print(Y_train.shape)

X_test, Y_test = normalize(test)
print(X_test.shape)
print(Y_test.shape)

# shuffle the training data since we do SGD
# we shuffle outside the training 
# since we want to compare unvectorized and vectorized versions
# It doesn't affect to batch training later
np.random.seed(8888)     # Do not change those seedings to make our results comparable
np.random.shuffle(train) 


(784, 4000)
(1, 4000)
(784, 500)
(1, 500)


# Unvectorized Version of SGD

First the unvectorized version of training:

In [5]:
def train_unvectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):   # Loop over every training instance
            # Forward pass
            z = 0 
            r = 0 # calculate the regularizer
            for i in range(n):
                z += w[i] * X_train[i,j]
                r += w[i]**2
            z += b
            o = 1. / (1 + math.exp(-z))

            # Calculate the loss
            c = (Y_train[:,j] - o)**2
            l = c / 2 + (lambdar / 2) * r
            L += l

            # Backward pass and update the weights/bias
            for i in range(n):
                w[i] -= lr * (o - Y_train[:,j]) * o * (1 - o) * X_train[i,j] + lambdar * w[i]
            b -= lr * (o - Y_train[:,j]) * o * (1 - o)
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    
    return w, b
        

And the (unvectorized) inference:

In [6]:
def test_unvectorized(X_test, Y_test, w, b):
    
    n_test = X_test.shape[0]
    m_test = X_test.shape[1]
    corrects = 0
    
    for j in range(m_test):

        # Forward pass
        z = 0 
        for i in range(n_test):
            z += w[i] * X_test[i,j]
        z += b
        o = 1. / (1 + np.exp(-z))
        # Evaluate the outputs
        if ((o >= 0.5) and (Y_test[0, j] == 1)) \
        or ((o < 0.5) and (Y_test[0, j] == 0)):
            corrects +=1
    
    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Test on our test data. The accuracy should be better than 89.2%. This high score 89.2% is the baseline, achieved by do nothing rather than predicting all images are not a "seven" :p.

In [7]:
w, b = train_unvectorized(X_train, Y_train)
_ = test_unvectorized(X_test, Y_test, w, b)

Error of the epoch 1: [ 0.01169449]
Error of the epoch 2: [ 0.00879146]
Error of the epoch 3: [ 0.00833166]
Error of the epoch 4: [ 0.00814921]
Error of the epoch 5: [ 0.00805634]
Accuracy of our LLSR:98.0%


# Vectorized Version of Stochastic Gradient Descent

Now we move to the vectorized version of training and inference, just replace for-loops and total-sums by $np.dot()$,  $np.sum()$ and the numpy pair-wise operations (you should do the vectorization using pens and papers first).

In [8]:
def train_vectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):

            # Forward pass
            z = np.dot(w, X_train[:,j]) + b
            r = np.sum(w**2)
            o = 1. / (1 + np.exp(-z))

            # Calculate the loss (for each instance - SGD) 
            c = (Y_train[:,j] - o)**2
            l = c / 2 + (lambdar / 2) * r
            L += l

            # Backward pass and update the weights/bias (for each instance - SGD) 
            w -= lr * (o - Y_train[:,j]) * o * (1 - o) * X_train[:,j] + lambdar * w 
            b -= lr * (o - Y_train[:,j]) * o * (1 - o)
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    return w, b

And the vectorized inference (short, clear and fast):

In [9]:
def test_vectorized(X_test, Y_test, w, b):
    
    m_test = X_test.shape[1]
    result = np.zeros((1, m_test))
    
    z = np.dot(w, X_test) + b
    o = 1. / (1 + np.exp(-z))
    result = (o > 0.5)
    corrects = np.sum(result == Y_test)

    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Those following runs should return exact the same outputs like the (unvectorized) training and inference before but in less than a second. The vectorization should be more effective (much faster) if this is not an one-layer logistic regression but a deep network.

In [10]:
w, b = train_vectorized(X_train, Y_train)
_ = test_vectorized(X_test, Y_test, w, b)

Error of the epoch 1: [ 0.01169449]
Error of the epoch 2: [ 0.00879146]
Error of the epoch 3: [ 0.00833166]
Error of the epoch 4: [ 0.00814921]
Error of the epoch 5: [ 0.00805634]
Accuracy of our LLSR:98.0%


# Vectorized Version of Batch Gradient Descent

Here is the fully vectorized version, batch training (vectorizing over training instances). The formula (you might be able to derive them after the next lecture):

$$ z = w \cdot X + b $$

$$ o = \sigma(z) $$

$$ C = \frac{1}{2m}\sum_{j=1}^{m}(y^{(j)}-o^{(j)})^2 $$

$$ R = \frac{1}{2m}\sum_{i=1}^{n}w_i^2 $$

$$ L = C + \lambda R $$

$$ \frac{\partial C}{\partial z^{(j)}} = \frac{1}{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ \frac{\partial z^{(j)}}{\partial w_i} = x_i $$

$$ \Rightarrow \frac{\partial C}{\partial w} = \frac{\partial C}{\partial z} \cdot X^T $$

$$ \frac{\partial R}{\partial w} = \frac{1}{m}w $$ 

$$ \Rightarrow \frac{\partial L}{\partial w} = \frac{\partial C}{\partial w} + \lambda\frac{\partial R}{\partial w} $$

$$ \frac{\partial z}{\partial b} = 1 $$

$$ \Rightarrow \frac{\partial L}{\partial b} = \frac{\partial C}{\partial b} = \sum_{j=1}^{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ w = w - \eta * \frac{\partial L}{\partial w} $$

$$ b = b - \eta *  \frac{\partial L}{\partial b} $$

In [11]:
def train_batch(X_train, Y_train, lr=0.1, lambdar=0.0001, epochs=50):
    
    n = X_train.shape[0]
    m = X_train.shape[1]

    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):

        # Forward pass
        z = np.dot(w, X_train) + b
        o = 1. / (1 + np.exp(-z))
        r = np.sum(w**2)    

        # Calculate the loss 
        # (axis here makes the training more general \
        # if there are more output neurons than 1, \
        # we want to sum over the instances, so axis=1)
        L = (np.sum((Y_train - o)**2, axis=1) + lambdar * r) / (2 * m)

        # Backward pass and update the weights/bias
        w -= lr * (np.dot((o - Y_train) * o * (1 - o), X_train.T) + lambdar * w) / m
        b -= lr * (np.sum((o - Y_train) * o * (1 - o))) / m

        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
        
    return w, b
        

Since it is a batch training and requires different hyperparameters, the result might not be comparable to the SGD trainings above. 

In [12]:
w_batch, b_batch = train_batch(X_train, Y_train, lr=2, lambdar=0.5, epochs=1000)
_ = test_vectorized(X_test, Y_test, w_batch, b_batch)

Error of the epoch 1: [ 0.16787075]
Error of the epoch 2: [ 0.05099794]
Error of the epoch 3: [ 0.0509903]
Error of the epoch 4: [ 0.05098236]
Error of the epoch 5: [ 0.05097411]
Error of the epoch 6: [ 0.05096553]
Error of the epoch 7: [ 0.0509566]
Error of the epoch 8: [ 0.05094729]
Error of the epoch 9: [ 0.05093758]
Error of the epoch 10: [ 0.05092744]
Error of the epoch 11: [ 0.05091684]
Error of the epoch 12: [ 0.05090575]
Error of the epoch 13: [ 0.05089413]
Error of the epoch 14: [ 0.05088193]
Error of the epoch 15: [ 0.05086912]
Error of the epoch 16: [ 0.05085564]
Error of the epoch 17: [ 0.05084144]
Error of the epoch 18: [ 0.05082646]
Error of the epoch 19: [ 0.05081062]
Error of the epoch 20: [ 0.05079386]
Error of the epoch 21: [ 0.05077608]
Error of the epoch 22: [ 0.05075718]
Error of the epoch 23: [ 0.05073706]
Error of the epoch 24: [ 0.05071559]
Error of the epoch 25: [ 0.05069262]
Error of the epoch 26: [ 0.05066798]
Error of the epoch 27: [ 0.05064149]
Error of the

Error of the epoch 242: [ 0.0080785]
Error of the epoch 243: [ 0.00807106]
Error of the epoch 244: [ 0.00806368]
Error of the epoch 245: [ 0.00805634]
Error of the epoch 246: [ 0.00804906]
Error of the epoch 247: [ 0.00804182]
Error of the epoch 248: [ 0.00803462]
Error of the epoch 249: [ 0.00802748]
Error of the epoch 250: [ 0.00802038]
Error of the epoch 251: [ 0.00801332]
Error of the epoch 252: [ 0.00800631]
Error of the epoch 253: [ 0.00799934]
Error of the epoch 254: [ 0.00799241]
Error of the epoch 255: [ 0.00798553]
Error of the epoch 256: [ 0.00797869]
Error of the epoch 257: [ 0.00797189]
Error of the epoch 258: [ 0.00796514]
Error of the epoch 259: [ 0.00795842]
Error of the epoch 260: [ 0.00795175]
Error of the epoch 261: [ 0.00794511]
Error of the epoch 262: [ 0.00793852]
Error of the epoch 263: [ 0.00793196]
Error of the epoch 264: [ 0.00792544]
Error of the epoch 265: [ 0.00791896]
Error of the epoch 266: [ 0.00791252]
Error of the epoch 267: [ 0.00790611]
Error of the 

Error of the epoch 461: [ 0.0070693]
Error of the epoch 462: [ 0.00706624]
Error of the epoch 463: [ 0.00706319]
Error of the epoch 464: [ 0.00706015]
Error of the epoch 465: [ 0.00705712]
Error of the epoch 466: [ 0.0070541]
Error of the epoch 467: [ 0.00705109]
Error of the epoch 468: [ 0.00704808]
Error of the epoch 469: [ 0.00704508]
Error of the epoch 470: [ 0.00704209]
Error of the epoch 471: [ 0.00703911]
Error of the epoch 472: [ 0.00703613]
Error of the epoch 473: [ 0.00703317]
Error of the epoch 474: [ 0.00703021]
Error of the epoch 475: [ 0.00702726]
Error of the epoch 476: [ 0.00702432]
Error of the epoch 477: [ 0.00702138]
Error of the epoch 478: [ 0.00701845]
Error of the epoch 479: [ 0.00701554]
Error of the epoch 480: [ 0.00701262]
Error of the epoch 481: [ 0.00700972]
Error of the epoch 482: [ 0.00700682]
Error of the epoch 483: [ 0.00700394]
Error of the epoch 484: [ 0.00700106]
Error of the epoch 485: [ 0.00699818]
Error of the epoch 486: [ 0.00699532]
Error of the e

Error of the epoch 680: [ 0.00655615]
Error of the epoch 681: [ 0.00655438]
Error of the epoch 682: [ 0.00655261]
Error of the epoch 683: [ 0.00655085]
Error of the epoch 684: [ 0.00654909]
Error of the epoch 685: [ 0.00654734]
Error of the epoch 686: [ 0.00654559]
Error of the epoch 687: [ 0.00654384]
Error of the epoch 688: [ 0.0065421]
Error of the epoch 689: [ 0.00654036]
Error of the epoch 690: [ 0.00653863]
Error of the epoch 691: [ 0.0065369]
Error of the epoch 692: [ 0.00653518]
Error of the epoch 693: [ 0.00653345]
Error of the epoch 694: [ 0.00653174]
Error of the epoch 695: [ 0.00653002]
Error of the epoch 696: [ 0.00652831]
Error of the epoch 697: [ 0.00652661]
Error of the epoch 698: [ 0.00652491]
Error of the epoch 699: [ 0.00652321]
Error of the epoch 700: [ 0.00652152]
Error of the epoch 701: [ 0.00651983]
Error of the epoch 702: [ 0.00651814]
Error of the epoch 703: [ 0.00651646]
Error of the epoch 704: [ 0.00651478]
Error of the epoch 705: [ 0.00651311]
Error of the e

Error of the epoch 901: [ 0.00624774]
Error of the epoch 902: [ 0.00624666]
Error of the epoch 903: [ 0.00624557]
Error of the epoch 904: [ 0.00624449]
Error of the epoch 905: [ 0.00624341]
Error of the epoch 906: [ 0.00624233]
Error of the epoch 907: [ 0.00624126]
Error of the epoch 908: [ 0.00624018]
Error of the epoch 909: [ 0.00623911]
Error of the epoch 910: [ 0.00623805]
Error of the epoch 911: [ 0.00623698]
Error of the epoch 912: [ 0.00623591]
Error of the epoch 913: [ 0.00623485]
Error of the epoch 914: [ 0.00623379]
Error of the epoch 915: [ 0.00623273]
Error of the epoch 916: [ 0.00623168]
Error of the epoch 917: [ 0.00623063]
Error of the epoch 918: [ 0.00622957]
Error of the epoch 919: [ 0.00622853]
Error of the epoch 920: [ 0.00622748]
Error of the epoch 921: [ 0.00622643]
Error of the epoch 922: [ 0.00622539]
Error of the epoch 923: [ 0.00622435]
Error of the epoch 924: [ 0.00622331]
Error of the epoch 925: [ 0.00622228]
Error of the epoch 926: [ 0.00622124]
Error of the

One thing to compare: the speed. Try to run the same number of epochs (1000) with SGD, vectorized training, you can see it still takes a long time to run compared to the fully batch training.

In [13]:
w, b = train_vectorized(X_train, Y_train, epochs=1000)
_ = test_vectorized(X_test, Y_test, w, b)

Error of the epoch 1: [ 0.01169449]
Error of the epoch 2: [ 0.00879146]
Error of the epoch 3: [ 0.00833166]
Error of the epoch 4: [ 0.00814921]
Error of the epoch 5: [ 0.00805634]
Error of the epoch 6: [ 0.00800113]
Error of the epoch 7: [ 0.00796388]
Error of the epoch 8: [ 0.00793627]
Error of the epoch 9: [ 0.00791453]
Error of the epoch 10: [ 0.00789692]
Error of the epoch 11: [ 0.00788258]
Error of the epoch 12: [ 0.00787101]
Error of the epoch 13: [ 0.00786176]
Error of the epoch 14: [ 0.00785442]
Error of the epoch 15: [ 0.00784861]
Error of the epoch 16: [ 0.00784401]
Error of the epoch 17: [ 0.00784036]
Error of the epoch 18: [ 0.00783746]
Error of the epoch 19: [ 0.00783513]
Error of the epoch 20: [ 0.00783327]
Error of the epoch 21: [ 0.00783176]
Error of the epoch 22: [ 0.00783053]
Error of the epoch 23: [ 0.00782954]
Error of the epoch 24: [ 0.00782872]
Error of the epoch 25: [ 0.00782806]
Error of the epoch 26: [ 0.00782751]
Error of the epoch 27: [ 0.00782705]
Error of t

Error of the epoch 220: [ 0.00782462]
Error of the epoch 221: [ 0.00782462]
Error of the epoch 222: [ 0.00782462]
Error of the epoch 223: [ 0.00782462]
Error of the epoch 224: [ 0.00782462]
Error of the epoch 225: [ 0.00782462]
Error of the epoch 226: [ 0.00782462]
Error of the epoch 227: [ 0.00782462]
Error of the epoch 228: [ 0.00782462]
Error of the epoch 229: [ 0.00782462]
Error of the epoch 230: [ 0.00782462]
Error of the epoch 231: [ 0.00782462]
Error of the epoch 232: [ 0.00782462]
Error of the epoch 233: [ 0.00782462]
Error of the epoch 234: [ 0.00782462]
Error of the epoch 235: [ 0.00782462]
Error of the epoch 236: [ 0.00782462]
Error of the epoch 237: [ 0.00782462]
Error of the epoch 238: [ 0.00782462]
Error of the epoch 239: [ 0.00782462]
Error of the epoch 240: [ 0.00782462]
Error of the epoch 241: [ 0.00782462]
Error of the epoch 242: [ 0.00782462]
Error of the epoch 243: [ 0.00782462]
Error of the epoch 244: [ 0.00782462]
Error of the epoch 245: [ 0.00782462]
Error of the

Error of the epoch 436: [ 0.00782462]
Error of the epoch 437: [ 0.00782462]
Error of the epoch 438: [ 0.00782462]
Error of the epoch 439: [ 0.00782462]
Error of the epoch 440: [ 0.00782462]
Error of the epoch 441: [ 0.00782462]
Error of the epoch 442: [ 0.00782462]
Error of the epoch 443: [ 0.00782462]
Error of the epoch 444: [ 0.00782462]
Error of the epoch 445: [ 0.00782462]
Error of the epoch 446: [ 0.00782462]
Error of the epoch 447: [ 0.00782462]
Error of the epoch 448: [ 0.00782462]
Error of the epoch 449: [ 0.00782462]
Error of the epoch 450: [ 0.00782462]
Error of the epoch 451: [ 0.00782462]
Error of the epoch 452: [ 0.00782462]
Error of the epoch 453: [ 0.00782462]
Error of the epoch 454: [ 0.00782462]
Error of the epoch 455: [ 0.00782462]
Error of the epoch 456: [ 0.00782462]
Error of the epoch 457: [ 0.00782462]
Error of the epoch 458: [ 0.00782462]
Error of the epoch 459: [ 0.00782462]
Error of the epoch 460: [ 0.00782462]
Error of the epoch 461: [ 0.00782462]
Error of the

Error of the epoch 652: [ 0.00782462]
Error of the epoch 653: [ 0.00782462]
Error of the epoch 654: [ 0.00782462]
Error of the epoch 655: [ 0.00782462]
Error of the epoch 656: [ 0.00782462]
Error of the epoch 657: [ 0.00782462]
Error of the epoch 658: [ 0.00782462]
Error of the epoch 659: [ 0.00782462]
Error of the epoch 660: [ 0.00782462]
Error of the epoch 661: [ 0.00782462]
Error of the epoch 662: [ 0.00782462]
Error of the epoch 663: [ 0.00782462]
Error of the epoch 664: [ 0.00782462]
Error of the epoch 665: [ 0.00782462]
Error of the epoch 666: [ 0.00782462]
Error of the epoch 667: [ 0.00782462]
Error of the epoch 668: [ 0.00782462]
Error of the epoch 669: [ 0.00782462]
Error of the epoch 670: [ 0.00782462]
Error of the epoch 671: [ 0.00782462]
Error of the epoch 672: [ 0.00782462]
Error of the epoch 673: [ 0.00782462]
Error of the epoch 674: [ 0.00782462]
Error of the epoch 675: [ 0.00782462]
Error of the epoch 676: [ 0.00782462]
Error of the epoch 677: [ 0.00782462]
Error of the

Error of the epoch 868: [ 0.00782462]
Error of the epoch 869: [ 0.00782462]
Error of the epoch 870: [ 0.00782462]
Error of the epoch 871: [ 0.00782462]
Error of the epoch 872: [ 0.00782462]
Error of the epoch 873: [ 0.00782462]
Error of the epoch 874: [ 0.00782462]
Error of the epoch 875: [ 0.00782462]
Error of the epoch 876: [ 0.00782462]
Error of the epoch 877: [ 0.00782462]
Error of the epoch 878: [ 0.00782462]
Error of the epoch 879: [ 0.00782462]
Error of the epoch 880: [ 0.00782462]
Error of the epoch 881: [ 0.00782462]
Error of the epoch 882: [ 0.00782462]
Error of the epoch 883: [ 0.00782462]
Error of the epoch 884: [ 0.00782462]
Error of the epoch 885: [ 0.00782462]
Error of the epoch 886: [ 0.00782462]
Error of the epoch 887: [ 0.00782462]
Error of the epoch 888: [ 0.00782462]
Error of the epoch 889: [ 0.00782462]
Error of the epoch 890: [ 0.00782462]
Error of the epoch 891: [ 0.00782462]
Error of the epoch 892: [ 0.00782462]
Error of the epoch 893: [ 0.00782462]
Error of the

# Vectorized Version of Minibatch Gradient Descent

Finally, we can do minibatch training, it is the same as batch training (the formula) but one iteration runs over a subset of the whole dataset at a time, and those subsets (minibatches) are shuffle before training. Note how I did that minibatch splitting and shuffling using our $numpy.random$ 

In [14]:
def train_minibatch(X_train, Y_train, batch_size=256, lr=0.1, lambdar=0.0001, epochs=50):
    
    n = X_train.shape[0]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        
        # Split into minibatches into a *list* of sub-arrays
        # we want to split along the number of instances, so axis = 1
        X_minibatch = np.array_split(X_train, batch_size, axis = 1)
        Y_minibatch = np.array_split(Y_train, batch_size, axis = 1) 
        
        # We shuffle the minibatches of X and Y in the same way
        nmb = len(X_minibatch) # number of minibatches
        np.random.seed(5678)
        shuffled_index = np.random.permutation(range(nmb))
                                               
        # Now we can do the training, we cannot vectorize over different minibatches
        # They are like our "epochs"
        for i in range(nmb):
            X_current = X_minibatch[shuffled_index[i]]
            Y_current = Y_minibatch[shuffled_index[i]]
            m = X_current.shape[1]

            # Forward pass
            z = np.dot(w, X_current) + b
            o = 1. / (1 + np.exp(-z))
            r = np.sum(w**2)    

            # Calculate the loss 
            # (axis here makes the training more general \
            # if there are more output neurons than 1, \
            # we want to sum over the instances, so axis=1)
            L = (np.sum((Y_current - o)**2, axis=1) + lambdar * r) / (2 * m)

            # Backward pass and update the weights/bias
            w -= lr * (np.dot((o - Y_current) * o * (1 - o), X_current.T) + lambdar * w) / m
            b -= lr * (np.sum((o - Y_current) * o * (1 - o))) / m

            print("Error of the iteration {0}: {1}".format(epoch * nmb + i + 1, L))

    return w, b

Minibatch Training for this LLSR is very sensitive to hyperparameter choosing. Should use with early stopping. Do not supprise if the accurary is bad. Shuffling the minibatch also takes time, so do not run this with large number of epochs.

In [15]:
# Do not run this for more than 100 epochs!!!!!!!!!
w_minibatch, b_minibatch = train_minibatch(X_train, Y_train, batch_size=512, lr=0.001, lambdar=0.0001, epochs=30)
_ = test_vectorized(X_test, Y_test, w_minibatch, b_minibatch)

Error of the iteration 1: [ 0.22118434]
Error of the iteration 2: [ 0.18991056]
Error of the iteration 3: [ 0.17519696]
Error of the iteration 4: [ 0.1285212]
Error of the iteration 5: [ 0.16185084]
Error of the iteration 6: [ 0.13454284]
Error of the iteration 7: [ 0.1802395]
Error of the iteration 8: [ 0.11435604]
Error of the iteration 9: [ 0.17015535]
Error of the iteration 10: [ 0.1701019]
Error of the iteration 11: [ 0.15010005]
Error of the iteration 12: [ 0.20916957]
Error of the iteration 13: [ 0.19987681]
Error of the iteration 14: [ 0.18505588]
Error of the iteration 15: [ 0.15651351]
Error of the iteration 16: [ 0.1017534]
Error of the iteration 17: [ 0.16357434]
Error of the iteration 18: [ 0.20742864]
Error of the iteration 19: [ 0.14822431]
Error of the iteration 20: [ 0.15424728]
Error of the iteration 21: [ 0.18803206]
Error of the iteration 22: [ 0.13032022]
Error of the iteration 23: [ 0.18788225]
Error of the iteration 24: [ 0.12726849]
Error of the iteration 25: [ 

Error of the iteration 250: [ 0.08968208]
Error of the iteration 251: [ 0.08155129]
Error of the iteration 252: [ 0.08918775]
Error of the iteration 253: [ 0.08612038]
Error of the iteration 254: [ 0.06277457]
Error of the iteration 255: [ 0.08795366]
Error of the iteration 256: [ 0.06463747]
Error of the iteration 257: [ 0.07456863]
Error of the iteration 258: [ 0.0781068]
Error of the iteration 259: [ 0.08439193]
Error of the iteration 260: [ 0.05260019]
Error of the iteration 261: [ 0.08434569]
Error of the iteration 262: [ 0.0811534]
Error of the iteration 263: [ 0.0860231]
Error of the iteration 264: [ 0.07848934]
Error of the iteration 265: [ 0.06669088]
Error of the iteration 266: [ 0.0945313]
Error of the iteration 267: [ 0.0896551]
Error of the iteration 268: [ 0.06096843]
Error of the iteration 269: [ 0.0827511]
Error of the iteration 270: [ 0.07473791]
Error of the iteration 271: [ 0.08018518]
Error of the iteration 272: [ 0.09286053]
Error of the iteration 273: [ 0.08995347

Error of the iteration 474: [ 0.0659444]
Error of the iteration 475: [ 0.07355355]
Error of the iteration 476: [ 0.06502651]
Error of the iteration 477: [ 0.06688964]
Error of the iteration 478: [ 0.04676305]
Error of the iteration 479: [ 0.0397493]
Error of the iteration 480: [ 0.04080333]
Error of the iteration 481: [ 0.03598856]
Error of the iteration 482: [ 0.03871897]
Error of the iteration 483: [ 0.04909463]
Error of the iteration 484: [ 0.05603706]
Error of the iteration 485: [ 0.05505691]
Error of the iteration 486: [ 0.06768379]
Error of the iteration 487: [ 0.0303859]
Error of the iteration 488: [ 0.05471758]
Error of the iteration 489: [ 0.05865526]
Error of the iteration 490: [ 0.03861934]
Error of the iteration 491: [ 0.03501444]
Error of the iteration 492: [ 0.04522479]
Error of the iteration 493: [ 0.06004609]
Error of the iteration 494: [ 0.02872449]
Error of the iteration 495: [ 0.06105205]
Error of the iteration 496: [ 0.05751334]
Error of the iteration 497: [ 0.03264

Error of the iteration 774: [ 0.03803408]
Error of the iteration 775: [ 0.04176573]
Error of the iteration 776: [ 0.04543178]
Error of the iteration 777: [ 0.031991]
Error of the iteration 778: [ 0.05735093]
Error of the iteration 779: [ 0.03854936]
Error of the iteration 780: [ 0.0460012]
Error of the iteration 781: [ 0.06593193]
Error of the iteration 782: [ 0.02757288]
Error of the iteration 783: [ 0.07155604]
Error of the iteration 784: [ 0.08761124]
Error of the iteration 785: [ 0.09567205]
Error of the iteration 786: [ 0.06042175]
Error of the iteration 787: [ 0.0578187]
Error of the iteration 788: [ 0.04998611]
Error of the iteration 789: [ 0.04725012]
Error of the iteration 790: [ 0.05560065]
Error of the iteration 791: [ 0.08234736]
Error of the iteration 792: [ 0.06712781]
Error of the iteration 793: [ 0.03448843]
Error of the iteration 794: [ 0.02059686]
Error of the iteration 795: [ 0.05042772]
Error of the iteration 796: [ 0.06706318]
Error of the iteration 797: [ 0.028323

Error of the iteration 1049: [ 0.05574375]
Error of the iteration 1050: [ 0.02688575]
Error of the iteration 1051: [ 0.02791786]
Error of the iteration 1052: [ 0.0514388]
Error of the iteration 1053: [ 0.04061332]
Error of the iteration 1054: [ 0.01721512]
Error of the iteration 1055: [ 0.05803768]
Error of the iteration 1056: [ 0.05097153]
Error of the iteration 1057: [ 0.01871926]
Error of the iteration 1058: [ 0.06206893]
Error of the iteration 1059: [ 0.0469395]
Error of the iteration 1060: [ 0.04934906]
Error of the iteration 1061: [ 0.02451768]
Error of the iteration 1062: [ 0.03367546]
Error of the iteration 1063: [ 0.03913892]
Error of the iteration 1064: [ 0.05642708]
Error of the iteration 1065: [ 0.03226333]
Error of the iteration 1066: [ 0.0477716]
Error of the iteration 1067: [ 0.04590862]
Error of the iteration 1068: [ 0.03621159]
Error of the iteration 1069: [ 0.04872028]
Error of the iteration 1070: [ 0.10706224]
Error of the iteration 1071: [ 0.07746799]
Error of the i

...

Error of the iteration 14285: [ 0.00580442]
Error of the iteration 14286: [ 0.03023911]
Error of the iteration 14287: [ 0.01554433]
Error of the iteration 14288: [ 0.01130777]
Error of the iteration 14289: [ 0.00432681]
Error of the iteration 14290: [ 0.02637632]
Error of the iteration 14291: [ 0.08290663]
Error of the iteration 14292: [ 0.03178034]
Error of the iteration 14293: [ 0.00919365]
Error of the iteration 14294: [ 0.00754239]
Error of the iteration 14295: [ 0.04518871]
Error of the iteration 14296: [ 0.02090487]
Error of the iteration 14297: [ 0.03467665]
Error of the iteration 14298: [ 0.02811689]
Error of the iteration 14299: [ 0.01952567]
Error of the iteration 14300: [ 0.02249085]
Error of the iteration 14301: [ 0.04221809]
Error of the iteration 14302: [ 0.01064128]
Error of the iteration 14303: [ 0.00852962]
Error of the iteration 14304: [ 0.00821552]
Error of the iteration 14305: [ 0.00633235]
Error of the iteration 14306: [ 0.00724807]
Error of the iteration 14307: [ 

Error of the iteration 14506: [ 0.00703571]
Error of the iteration 14507: [ 0.04933704]
Error of the iteration 14508: [ 0.01399931]
Error of the iteration 14509: [ 0.00682548]
Error of the iteration 14510: [ 0.00425972]
Error of the iteration 14511: [ 0.00837354]
Error of the iteration 14512: [ 0.00461351]
Error of the iteration 14513: [ 0.02162908]
Error of the iteration 14514: [ 0.01450128]
Error of the iteration 14515: [ 0.00894958]
Error of the iteration 14516: [ 0.00828586]
Error of the iteration 14517: [ 0.01610426]
Error of the iteration 14518: [ 0.023744]
Error of the iteration 14519: [ 0.02338236]
Error of the iteration 14520: [ 0.00732843]
Error of the iteration 14521: [ 0.02208381]
Error of the iteration 14522: [ 0.01756587]
Error of the iteration 14523: [ 0.05995671]
Error of the iteration 14524: [ 0.01887786]
Error of the iteration 14525: [ 0.02094576]
Error of the iteration 14526: [ 0.02209319]
Error of the iteration 14527: [ 0.00429366]
Error of the iteration 14528: [ 0.

Error of the iteration 14706: [ 0.02776528]
Error of the iteration 14707: [ 0.00093266]
Error of the iteration 14708: [ 0.00419116]
Error of the iteration 14709: [ 0.00704408]
Error of the iteration 14710: [ 0.02681334]
Error of the iteration 14711: [ 0.00950304]
Error of the iteration 14712: [ 0.00637111]
Error of the iteration 14713: [ 0.00768764]
Error of the iteration 14714: [ 0.01124676]
Error of the iteration 14715: [ 0.00544355]
Error of the iteration 14716: [ 0.01760975]
Error of the iteration 14717: [ 0.00565312]
Error of the iteration 14718: [ 0.00997161]
Error of the iteration 14719: [ 0.02924802]
Error of the iteration 14720: [ 0.00643207]
Error of the iteration 14721: [ 0.03303874]
Error of the iteration 14722: [ 0.03006743]
Error of the iteration 14723: [ 0.01169922]
Error of the iteration 14724: [ 0.00664573]
Error of the iteration 14725: [ 0.0308326]
Error of the iteration 14726: [ 0.00326567]
Error of the iteration 14727: [ 0.01935614]
Error of the iteration 14728: [ 0

Error of the iteration 14991: [ 0.03137746]
Error of the iteration 14992: [ 0.01693566]
Error of the iteration 14993: [ 0.00243588]
Error of the iteration 14994: [ 0.00464145]
Error of the iteration 14995: [ 0.00919291]
Error of the iteration 14996: [ 0.01119483]
Error of the iteration 14997: [ 0.00457996]
Error of the iteration 14998: [ 0.0519725]
Error of the iteration 14999: [ 0.0153023]
Error of the iteration 15000: [ 0.03635308]
Error of the iteration 15001: [ 0.01008824]
Error of the iteration 15002: [ 0.00243104]
Error of the iteration 15003: [ 0.00497559]
Error of the iteration 15004: [ 0.01000319]
Error of the iteration 15005: [ 0.00772868]
Error of the iteration 15006: [ 0.0046621]
Error of the iteration 15007: [ 0.01539416]
Error of the iteration 15008: [ 0.01579102]
Error of the iteration 15009: [ 0.00111665]
Error of the iteration 15010: [ 0.00322603]
Error of the iteration 15011: [ 0.00168024]
Error of the iteration 15012: [ 0.01623763]
Error of the iteration 15013: [ 0.0

Error of the iteration 15197: [ 0.06211922]
Error of the iteration 15198: [ 0.02543168]
Error of the iteration 15199: [ 0.00870118]
Error of the iteration 15200: [ 0.00572042]
Error of the iteration 15201: [ 0.0076892]
Error of the iteration 15202: [ 0.03143093]
Error of the iteration 15203: [ 0.03793338]
Error of the iteration 15204: [ 0.00665099]
Error of the iteration 15205: [ 0.01141737]
Error of the iteration 15206: [ 0.05078212]
Error of the iteration 15207: [ 0.00529844]
Error of the iteration 15208: [ 0.01134725]
Error of the iteration 15209: [ 0.01370666]
Error of the iteration 15210: [ 0.00271335]
Error of the iteration 15211: [ 0.0423166]
Error of the iteration 15212: [ 0.0086673]
Error of the iteration 15213: [ 0.0360068]
Error of the iteration 15214: [ 0.01303177]
Error of the iteration 15215: [ 0.02112734]
Error of the iteration 15216: [ 0.00778248]
Error of the iteration 15217: [ 0.00100117]
Error of the iteration 15218: [ 0.02754141]
Error of the iteration 15219: [ 0.00